In [1]:
import json
from pprint import pprint
import os
import sqlite3
from __future__ import print_function
import unicodedata
import datetime
import numpy as np

In [7]:
pathToDB = "/root/jupyter_NB/packettrace-DB-1.db"
conn = sqlite3.connect(pathToDB)
c = conn.cursor()

In [11]:
# returns events, that went from source to dest as a list
# the list is ordered in time, where the newest event comes first
def getDataPoints(coursor):
    dataStrings = []
    counter = 0
    for row in coursor.execute('SELECT datapoints FROM rawdata ORDER BY event_timestamp DESC'):
        counter += 1
        dataStrings.append(row)
    print(str(counter) + " Datapoints were read.")
    # store all datapoints in a fitting format
    dataPoints = []
    for point in dataStrings:
        # do unicode converstion and formatting
        text = point[0].encode('ascii','ignore').replace("u", '').replace("'", '"')
        data = json.loads(text)
        dataPoints.append(data)
    return dataPoints

# datetime objects are simpler to handle than timestamps and they support comparrison
def convertTimestampToDatetime(timestamp):
    return datetime.datetime.fromtimestamp(timestamp)

def avageAndVarianceOverWeeks(dataPointsArray, currentIndex, averagingWeeks):
    outTupel = (0, 0)
    weekInSeconds = 604800
    weeksAgo = dataPointsArray[currentIndex, 0] - (weekInSeconds * averagingWeeks)
    if (dataPointsArray[len(dataPointsArray)-1, 0] > weeksAgo):
        weeksAgo = dataPointsArray[len(dataPointsArray)-1, 0]
    # cut array
    lastIndex = 0
    for i in xrange(len(dataPointsArray)):
        if dataPointsArray[i, 0] <= weeksAgo :
            lastIndex = i
    # TODO: CUT THE ACTUAL ARRAY AND CONTIUNE WITH AVERAGING AND VAIANZ STUFF
    dataSum = 0
    for point in dataPoints:
        break
        
    

def createThroughputDataset(dataPoints, averagingWeeks):
    # since we know, that we only have one datapoint per listentry we will convert the list to a numpy array
    dataPointsArray = np.empty([len(dataPoints),2], np.int32)
    for i in xrange(len(dataPointsArray)):
        dataPointsArray[i,0] = int(dataPoints[i].keys()[0])
        dataPointsArray[i,1] = int(dataPoints[i].values()[0])
    # create a Element in the List for each DataPoint
    for i in xrange(len(dataPointsArray)):
        currentSet = {}
        currentSet["timestamp"] = dataPointsArray[i,0]
        currentSet["measurement"] = dataPointsArray[i,1]
        avgAVar = avageAndVarianceOverWeeks(dataPointsArray, i, averagingWeeks)
        currentSet["average"] = avgAVar[0]
        currentSet["variance"] = avgAVar[1]
        
    

In [ ]:
dataPoints = getDataPoints(c)